In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
class SyntheticDataset(tf.keras.utils.Sequence):
    def __init__(self, num_samples, input_dim, output_dim, batch_size):
        self.num_samples = num_samples
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.batch_size = batch_size

        # Generate synthetic data
        self.inputs = np.random.rand(self.num_samples, self.input_dim).astype(
            np.float32
        )
        self.targets = np.random.rand(self.num_samples, self.output_dim).astype(
            np.float32
        )

    def __len__(self):
        return int(np.ceil(self.num_samples / self.batch_size))

    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = min((idx + 1) * self.batch_size, self.num_samples)
        batch_inputs = self.inputs[start_idx:end_idx]
        batch_targets = self.targets[start_idx:end_idx]
        return batch_inputs, batch_targets

In [ ]:
# Example usage:
batch_size = 32
dataset = SyntheticDataset(
    num_samples=1000, input_dim=10, output_dim=1, batch_size=batch_size
)

dataloader = tf.data.Dataset.from_tensor_slices((dataset.inputs, dataset.targets))
dataloader = dataloader.shuffle(buffer_size=len(dataset)).batch(batch_size)
for inputs, targets in dataloader:
    print(inputs, targets)